In [32]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices("GPU"))

TensorFlow Version: 2.17.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [33]:
# Load datasets
print("🔄 Loading data...")
train_data = pd.read_csv("data/TEP_FaultFree_Training.csv")
test_data = pd.read_csv("data/TEP_FaultFree_Testing.csv")

# Drop index column if necessary
train_data = train_data.iloc[:, 1:]
test_data = test_data.iloc[:, 1:]

🔄 Loading data...


In [34]:
# Normalize data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train_data)
test_scaled = scaler.transform(test_data)

In [35]:
input_dim = train_scaled.shape[1]
input_layer = keras.Input(shape=(input_dim,))
encoded = layers.Dense(128, activation='relu')(input_layer)
encoded = layers.Dense(64, activation='relu')(encoded)
encoded = layers.Dense(32, activation='relu', activity_regularizer=regularizers.l1(1e-4))(encoded)

decoded = layers.Dense(64, activation='relu')(encoded)
decoded = layers.Dense(128, activation='relu')(decoded)
decoded = layers.Dense(input_dim, activation='sigmoid')(decoded)

In [ ]:
# Compile model
autoencoder = keras.Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

# Encoder model
encoder = keras.Model(input_layer, encoded)

# Train autoencoder
print("🚀 Training autoencoder...")
history = autoencoder.fit(
    train_scaled, train_scaled,
    epochs=50,
    batch_size=32,
    validation_data=(test_scaled, test_scaled),
    verbose=1
)

🚀 Training autoencoder...
Epoch 1/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 116s 15ms/step - loss: 0.0161 - val_loss: 0.0243
Epoch 2/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 112s 14ms/step - loss: 0.0160 - val_loss: 0.0243
Epoch 3/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 116s 15ms/step - loss: 0.0160 - val_loss: 0.0243
Epoch 4/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 126s 16ms/step - loss: 0.0160 - val_loss: 0.0243
Epoch 5/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 122s 16ms/step - loss: 0.0160 - val_loss: 0.0243
Epoch 6/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 119s 15ms/step - loss: 0.0160 - val_loss: 0.0243
Epoch 7/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 130s 17ms/step - loss: 0.0160 - val_loss: 0.0242
Epoch 8/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 133s 17ms/step - loss: 0.0160 - val_loss: 0.0243
Epoch 9/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 137s 18ms/step - loss: 0.0160 - val_loss: 0.0242
Epoch 10/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 129s 17ms/step - loss: 0.0160 - val_loss: 0.0242
Epoch 11/50
7813/7813 ━━━━━━━━━━━━━━━━━━━━ 127s 16ms/step - los

In [ ]:
# Save model and scaler
os.makedirs("models", exist_ok=True)
autoencoder.save("models/autoencoder_dip.h5")
np.save("models/scaler.npy", scaler)
print("✅ Model trained and saved!")

In [ ]:
# Extract latent space representation
latent_rep = encoder.predict(test_scaled)

# t-SNE visualization
tsne = TSNE(n_components=2, random_state=42)
tsne_results = tsne.fit_transform(latent_rep)
plt.figure(figsize=(8, 6))
plt.scatter(tsne_results[:, 0], tsne_results[:, 1], alpha=0.5)
plt.title('t-SNE of Latent Space')
plt.show()

# Reconstruction error analysis
reconstructions = autoencoder.predict(test_scaled)
recon_errors = np.mean(np.square(test_scaled - reconstructions), axis=1)
plt.figure(figsize=(8, 6))
sns.histplot(recon_errors, kde=True, stat='probability')
plt.axvline(np.percentile(recon_errors, 95), color='r', linestyle='--')  # 95% threshold
plt.title('Reconstruction Error Distribution')
plt.show()